<a href="https://colab.research.google.com/github/luuleitner/dasIT/blob/main/beamform_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Beamforming Tutorial</h1>

This is a hands-on introduction to ultrasound beamforming. During this exercise we will look at the raw ultrasound data acquired and how to convert it into actual images. This practical example is part of the Graz University of Technology lecture series *Development of Electronic Systems* and the *Fundamentals of Biomedical Engineering Laboratory*.

I wish everyone a great dive into the topic, and please do not hesitate to <a href="mailto:christoph.leitner@tugraz.at">contact</a> me in case of **any** questions!

yours,<br>
Christoph<br><br>

<h4>Free Ultrasound Ressources:</h4>

*   <a href="http://www.k-wave.org/">k-wave ultrasound simulator</a> - free MATLAB and C++ implementations
*   <a href="https://field-ii.dk/">field II ultrasound simulator</a> - free MATLAB implementation
*   <a href="https://github.com/Sergio5714/pybf">pybf - Python beamformer</a> - optimized for short processing times
*   <a href="https://www.biomecardio.com/MUST/">MATLAB ultrasound toolbox</a> - free MATLAB beamformer<br><br>


<h4>Contact</h4>
Christoph Leitner<br>
e <a href="mailto:christoph.leitner@tugraz.at">christoph.leitner@tugraz.at</a><br>
g <a href="https://github.com/luuleitner/dasIT">github.com/luuleitner/dasIT</a><br><br>

---

<h2>Getting Started</h2>


```
[#] Areas shown like this are executable code. Use the mousover play button to run these cells.
```


First we need to install the beamformer package from the the <a href="https://github.com/luuleitner/dasIT">GitHub repository</a>:

In [ ]:
# Fetch the newest dasIT package from the github repository

!pip install git+https://github.com/luuleitner/dasIT

Next we load all other necessary libraries into our Colab notebook:

In [ ]:
# import necessary packages
import os
import numpy as np
from datetime import datetime
from matplotlib import pyplot as plt

# to run dasIT, we need to import the necessary commands from the library
from dasIT.data.loader import RFDataloader, TDloader, TGCloader
from dasIT.features.transducer import transducer
from dasIT.features.medium import medium
from dasIT.features.tgc import tg_compensation
from dasIT.src.delays import planewave_delays
from dasIT.src.apodization import apodization
from dasIT.src.das_bf import RXbeamformer
from dasIT.features.signal import RFfilter, fftsignal, analytic_signal
from dasIT.features.image import interp_lateral
from dasIT.visualization.signal_callback import amp_freq_1channel, amp_1channel
from dasIT.visualization.image_callback import plot_signal_grid, plot_signal_image

###Download the example dataset

In [ ]:
# import 5 RF-data frames captured on a Verasonics Vantage 256 system*, using a GE-9LD transducer, and a CIRS generalpurpose ultrasound phantom.
#
# * Leitner et al. 2020, "Detection of Motor Endplates in Deep and Pennate Skeletal Muscles in-vivo using Ultrafast Ultrasound",
# 2020 IEEE International Ultrasonics Symposium (IUS).
#

rfdata_path = '/content/rfdata'

if os.path.exists(rfdata_path) == False:
  os.mkdir(rfdata_path)
  os.chdir(rfdata_path)
  !wget -i https://raw.githubusercontent.com/luuleitner/dasIT/main/example_data/CIRSphantom_GE9LD_VVantage/COLABdownload_url.txt

os.chdir(rfdata_path)

#dasIT Transducer and Medium

In [ ]:
# dasIT transducer
physical_transducer = TDloader('transducer.csv')
dasIT_transducer = transducer(center_frequency_hz=physical_transducer.transducer['center frequency'].dropna().to_numpy(dtype='float', copy=False),  # [Hz]
                              bandwidth_hz=physical_transducer.transducer['bandwidth'].dropna().to_numpy(dtype='float', copy=False),    # [Hz]
                              adc_ratio=4,  # [-]
                              transducer_elements_nr=physical_transducer.transducer['number of elements'].dropna().to_numpy(dtype='float', copy=False), # [#]
                              element_pitch_m=physical_transducer.transducer['element pitch'].dropna().to_numpy(dtype='float', copy=False), # [m]
                              pinmap=physical_transducer.transducer['pinmap'].dropna().to_numpy(dtype='int', copy=False),   # [-]
                              pinmapbase=1, # [-]
                              elevation_focus=0.028, # [m]
                              focus_number=None,
                              totalnr_planewaves=1,     # [-]
                              planewave_angle_interval=[0,0],   # [rad]
                              axial_cutoff_wavelength=5,  # [#]
                              speed_of_sound_ms = 1540)  # [m/s]

print(f'Transducer properties:')
print()
vars(dasIT_transducer)

In [ ]:
# dasIT medium
dasIT_medium = medium(speed_of_sound_ms = 1540, # [m/s]
                      center_frequency = dasIT_transducer.center_frequency, # [Hz]
                      sampling_frequency = dasIT_transducer.sampling_frequency, # [Hz]
                      max_depth_wavelength = 177,   # [#]
                      lateral_transducer_element_spacing = dasIT_transducer.lateral_transducer_spacing, # [m]
                      axial_extrapolation_coef = 1.05,  # [-]
                      attenuation_coefficient= 0.75,   # [dB/(MHz^y cm)]
                      attenuation_power=1.5   # [-]
                      )

#Load and Preprocess RF-data

In [ ]:
### Load RF Data
RFdata = RFDataloader('CIRS_phantom.h5')

### Preprocess (Clip and Sort) RF Data
# Samples start: at first recorded echo (number of wavelength distance is provided from vendor)
# -> null out the rest to not overshadow the real results
# Samples end: at penetration depth -> clip rest of samples without data
# If necessary sort the transducer channels according to the pin map to get the channels first-last channel
RFdata.signal[:dasIT_transducer.start_depth_rec_samples, :, :] = 0
RFdata.signal = RFdata.signal[:dasIT_medium.rx_echo_totalnr_samples, dasIT_transducer.transducer_pinmap, :]

print(f'Channels of transducer: {RFdata.signal.shape[1]}')
print(f'Samples per channel: {RFdata.signal.shape[0]}')
print(f'Number of frames: {RFdata.signal.shape[2]}')

In [ ]:
# Plot channel 156
channel =156
fig = plt.figure(figsize=(5, 1), dpi=100)
ax_1 = fig.add_subplot(111)
ax_1.plot(RFdata.signal[:,channel,0])
ax_1.set_xlabel('Samples [#]')
ax_1.set_ylabel('Signal [V]')
ax_1.set_title(f'RF-data channel {channel}')

#Signal Preprocessing

###Time Gain Compensation

In [ ]:
# Load tgc-waveform
tgc_cntrl_points = TGCloader(controlpt_path='tgc_cntrl_pt.csv')
# Apply TGC
TGCsignals = tg_compensation(signals=RFdata.signal,
                             medium=dasIT_medium,
                             center_frequency=dasIT_transducer.center_frequency,
                             cntrl_points=tgc_cntrl_points,
                             mode='points')

In [ ]:
# Plot channel 156
fig = plt.figure(figsize=(5, 3), dpi=100)
ax_1 = fig.add_subplot(211)
ax_1.plot(RFdata.signal[:,channel,0])
ax_1.set_xlabel('Samples [#]')
ax_1.set_ylabel('Signal [V]')
ax_1.set_title(f'RF-data channel {channel}')

ax_2 = fig.add_subplot(212)
ax_2.plot(TGCsignals.signals[:,channel,0],'r')
ax_2.set_xlabel('Samples [#]')
ax_2.set_ylabel('Signal [V]')
ax_2.set_title(f'TGC RF-data channel {channel}')

plt.tight_layout()
plt.show()

###Filtering

In [ ]:
### Filter RF Data
RFdata_filtered = RFfilter(signals=TGCsignals.signals,
                           fcutoff_band=dasIT_transducer.bandwidth,
                           fsampling=dasIT_transducer.sampling_frequency,
                           type='gaussian',
                           order=10)

In [ ]:
# Plot channel 156
fftFil = fftsignal(RFdata_filtered.signal[:,channel,0], dasIT_transducer.sampling_frequency)

fig = plt.figure(figsize=(6, 5), dpi=100)
ax_1 = fig.add_subplot(311)
ax_1.plot(RFdata.signal[:,channel,0])
ax_1.set_xlabel('Samples [#]')
ax_1.set_ylabel('Signal [V]')
ax_1.set_title(f'RF-data channel {channel}')

ax_2 = fig.add_subplot(312)
ax_2.plot(RFdata_filtered.signal[:,channel,0],'r')
ax_2.set_xlabel('Samples [#]')
ax_2.set_ylabel('Signal [V]')
ax_2.set_title(f'Filtered RF-data channel {channel}')
 
ax_3 = fig.add_subplot(313)
ax_3.plot(fftFil[0], fftFil[1], 'r')
ax_3.set_xlabel('Frequency [MHz]')
ax_3.set_ylabel('Power [W/Hz]')
ax_3.set_title(f'FFT channel {channel}')

plt.tight_layout()
plt.show()

###Convert to analytical signal

In [ ]:
####################################################################
#------------------------ Analytical Signal -----------------------#

### Hilbert Transform
RFdata_analytic = analytic_signal(np.squeeze(RFdata_filtered.signal), interp=False)

In [ ]:
RFdata_analytic[:,channel,0].imag

In [ ]:
# Plot channel 156
fftFil = fftsignal(RFdata_filtered.signal[:,channel,0], dasIT_transducer.sampling_frequency)

fig = plt.figure(figsize=(5, 3), dpi=100)
ax_1 = fig.add_subplot(211)
ax_1.plot(RFdata.signal[:,channel,0])
ax_1.set_xlabel('Samples [#]')
ax_1.set_ylabel('Signal [V]')
ax_1.set_title(f'RF-data channel {channel}')

ax_2 = fig.add_subplot(212)
ax_2.plot(RFdata_analytic[:,channel,0].real,'r')
ax_2.plot(abs(RFdata_analytic[:,channel,0]),'g')
ax_2.set_xlabel('Samples [#]')
ax_2.set_ylabel('Signal [V]')
ax_2.set_title(f'Analytic signal channel {channel}')

plt.tight_layout()
plt.show()

#Beamforming

###Element directivity and focus

In [ ]:
####################################################################
#-------------------------- Apodization Table --------------------------#

apodization = apodization(delays=None,
                          medium=dasIT_medium.medium,
                          transducer=dasIT_transducer,
                          apo='rec',
                          angles=dasIT_transducer.planewave_angles())

###Delay tables

In [ ]:
####################################################################
#-------------------------- Delay Tables --------------------------#

### DAS delay tabels for tilted planewaves
delay_table = planewave_delays(medium=dasIT_medium.medium,
                               sos=dasIT_medium.speed_of_sound,
                               fsampling=dasIT_transducer.sampling_frequency,
                               angles=dasIT_transducer.planewave_angles())

###Beamforming

In [ ]:
####################################################################
#-------------------------- Beamforming ---------------------------#
start_das_timing = datetime.now()

# Mask images areas in axial direction which have been included for reconstruction
# but are not part of the actual image.
RFsignals = RFdata_analytic[:,:,0]

RFsignals = np.expand_dims(RFsignals, 2)
RFsignals = np.repeat(RFsignals, RFsignals.shape[1], axis=2)
RFsignals = np.expand_dims(RFsignals, 3)

BFsignals = RXbeamformer(signals=RFsignals,
                         delays=delay_table.sample_delays,
                         apodization=apodization.table)

#Image Formation

In [ ]:
####################################################################
#------------------------ Image Formation --------------------------

# Envelope
BFsignals.envelope = abs(BFsignals.frame)

# Interpolate over Lateral space
BFsignals.interpolated = interp_lateral(signals=BFsignals.envelope,
                                        transducer=dasIT_transducer,
                                        medium=dasIT_medium,
                                        scale=3)


# Plot image
plot_signal_grid(signals=BFsignals.interpolated.signals_lateral_interp,
                 axis_vectors_xz=BFsignals.interpolated.imagegrid_mm,
                 axial_clip=[dasIT_transducer.start_depth_rec_m, None],
                 compression=True,
                 dbrange=58)

In [ ]:
####################################################################
#------------------------ Image Formation --------------------------

# Envelope
BFsignals.envelope = abs(BFsignals.frame)

# Interpolate over Lateral space
BFsignals.interpolated = interp_lateral(signals=BFsignals.envelope,
                                        transducer=dasIT_transducer,
                                        medium=dasIT_medium,
                                        scale=3)


# Plot image
plot_signal_grid(signals=BFsignals.interpolated.signals_lateral_interp,
                 axis_vectors_xz=BFsignals.interpolated.imagegrid_mm,
                 axial_clip=[dasIT_transducer.start_depth_rec_m, None],
                 compression=True,
                 dbrange=58)